# Vehicle Router - Optimization Examples

This notebook demonstrates how to use the Vehicle Router optimizers with practical examples.

## Table of Contents
1. Setup and Imports
2. Basic Example
3. Standard MILP + Greedy
4. Enhanced MILP
5. Genetic Algorithm
6. Custom Data Example
7. Performance Comparison
8. Visualization


## Setup and Imports

First, let's import the necessary modules and set up the environment.


In [1]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from pathlib import Path

# Add project root to Python path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Import Vehicle Router modules
from vehicle_router.optimizer import VrpOptimizer
from vehicle_router.enhanced_optimizer import EnhancedVrpOptimizer
from vehicle_router.genetic_optimizer import GeneticVrpOptimizer
from vehicle_router.data_generator import DataGenerator
from vehicle_router.utils import calculate_distance_matrix
from vehicle_router.plotting import plot_routes, plot_costs, plot_utilization

print("✅ All imports successful!")
print(f"📁 Project root: {project_root}")


✅ All imports successful!
📁 Project root: c:\Local\Vehicle_Router


## Basic Example

Let's start with a simple example using the default Standard MILP + Greedy optimizer.


In [2]:
# Generate example data
print("🔄 Generating example data...")
generator = DataGenerator(use_example_data=True, random_seed=42)

orders_df = generator.generate_orders()
trucks_df = generator.generate_trucks()

# Get unique postal codes for distance matrix
postal_codes = orders_df['postal_code'].unique().tolist()
postal_codes.append('08020')  # Add depot

distance_matrix = generator.generate_distance_matrix(postal_codes)

print(f"📦 Generated {len(orders_df)} orders")
print(f"🚛 Generated {len(trucks_df)} trucks")
print(f"🗺️ Distance matrix: {distance_matrix.shape[0]}x{distance_matrix.shape[1]}")

# Display the data
print("\n📋 Orders:")
display(orders_df)

print("\n🚛 Trucks:")
display(trucks_df)


[INFO] DataGenerator initialized with use_example_data=True
[INFO] Starting order data generation...
[INFO] Generated 5 example orders with total volume 150.0 m³
[INFO] Order details:
[INFO]   Order A: 25.0 m³ at postal code 08031
[INFO]   Order B: 50.0 m³ at postal code 08030
[INFO]   Order C: 25.0 m³ at postal code 08029
[INFO]   Order D: 25.0 m³ at postal code 08028
[INFO]   Order E: 25.0 m³ at postal code 08027
[INFO] Starting truck data generation...
[INFO] Generated 5 example trucks with total capacity 225.0 m³
[INFO] Truck details:
[INFO]   Truck 1.0: 100.0 m³ capacity, €1500.0 cost
[INFO]   Truck 2.0: 50.0 m³ capacity, €1000.0 cost
[INFO]   Truck 3.0: 25.0 m³ capacity, €500.0 cost
[INFO]   Truck 4.0: 25.0 m³ capacity, €1500.0 cost
[INFO]   Truck 5.0: 25.0 m³ capacity, €1000.0 cost
[INFO] Processing 6 unique postal codes: ['08020', '08027', '08028', '08029', '08030', '08031']
[INFO] Using simulated distances (1km per postal code unit)
[INFO] Generated 6x6 distance matrix
[INFO] 

🔄 Generating example data...
📦 Generated 5 orders
🚛 Generated 5 trucks
🗺️ Distance matrix: 6x6

📋 Orders:


,order_id,volume,postal_code
0,A,25.0,08031
1,B,50.0,08030
2,C,25.0,08029
3,D,25.0,08028
4,E,25.0,08027



🚛 Trucks:


,truck_id,capacity,cost
0,1,100.0,1500.0
1,2,50.0,1000.0
2,3,25.0,500.0
3,4,25.0,1500.0
4,5,25.0,1000.0


## Standard MILP + Greedy

The Standard MILP + Greedy optimizer uses a two-phase approach: MILP for truck selection and greedy algorithm for route optimization.


In [3]:
print("🚀 Running Standard MILP + Greedy optimization...")

# Create optimizer
optimizer = VrpOptimizer(
    orders_df=orders_df,
    trucks_df=trucks_df,
    distance_matrix=distance_matrix,
    depot_location='08020',
    depot_return=False,
    max_orders_per_truck=3,
    enable_greedy_routes=True
)

# Build and solve
start_time = time.time()
optimizer.build_model()
success = optimizer.solve(timeout=300)
solve_time = time.time() - start_time

if success:
    solution = optimizer.get_solution()
    print(f"✅ Optimization successful! ({solve_time:.2f}s)")
    print(f"💰 Total cost: €{solution['costs']['total_cost']:.0f}")
    print(f"🚛 Trucks used: {len(solution['selected_trucks'])}")
    print(f"📦 Orders delivered: {len(solution['assignments_df'])}")
    
    # Display solution summary
    print("\n📊 Solution Summary:")
    print(optimizer.get_solution_summary_text())
else:
    print("❌ Optimization failed")


[INFO] Initializing VRP Optimizer...
[INFO] Input data validation completed successfully
[INFO] Depot location: 08020
[INFO] Depot return enabled: False
[INFO] Greedy route optimization enabled: True
[INFO] Maximum orders per truck: 3
[INFO] Optimizer initialized with 5 orders and 5 trucks
[INFO] Total order volume: 150.0 m³
[INFO] Total truck capacity: 225.0 m³
[INFO] Building MILP optimization model...
[INFO] Created 25 assignment variables
[INFO] Created 5 truck usage variables
[INFO] Objective function set to minimize total truck costs
[INFO] Adding order assignment constraints...
[INFO] Added 5 order assignment constraints
[INFO] Adding capacity constraints...
[INFO] Added capacity constraint for Truck 1: max 100.0 m³
[INFO] Added capacity constraint for Truck 2: max 50.0 m³
[INFO] Added capacity constraint for Truck 3: max 25.0 m³
[INFO] Added capacity constraint for Truck 4: max 25.0 m³
[INFO] Added capacity constraint for Truck 5: max 25.0 m³
[INFO] Added 5 capacity constraints

🚀 Running Standard MILP + Greedy optimization...
✅ Optimization successful! (0.07s)
💰 Total cost: €2500
🚛 Trucks used: 2
📦 Orders delivered: 5

📊 Solution Summary:
=== VEHICLE ROUTER ===
Selected Trucks: [1, 2]
Truck 1 -> Orders ['A', 'B', 'D']
Truck 2 -> Orders ['C', 'E']
Total Cost: €2500


## Enhanced MILP

The Enhanced MILP optimizer provides multi-objective optimization with cost and distance weighting.


In [4]:
print("🚀 Running Enhanced MILP optimization...")

# Create enhanced optimizer
enhanced_optimizer = EnhancedVrpOptimizer(
    orders_df=orders_df,
    trucks_df=trucks_df,
    distance_matrix=distance_matrix,
    depot_location='08020',
    depot_return=False,
    max_orders_per_truck=3
)

# Set cost and distance weights
enhanced_optimizer.set_weights(cost_weight=0.7, distance_weight=0.3)

# Build and solve
start_time = time.time()
enhanced_optimizer.build_model()
success = enhanced_optimizer.solve(timeout=300)
solve_time = time.time() - start_time

if success:
    enhanced_solution = enhanced_optimizer.get_solution()
    print(f"✅ Enhanced optimization successful! ({solve_time:.2f}s)")
    print(f"💰 Total cost: €{enhanced_solution['costs']['total_cost']:.0f}")
    print(f"🚛 Trucks used: {len(enhanced_solution['selected_trucks'])}")
    print(f"📦 Orders delivered: {len(enhanced_solution['assignments_df'])}")
    
    # Display solution summary
    print("\n📊 Enhanced Solution Summary:")
    print(enhanced_optimizer.get_solution_summary_text())
else:
    print("❌ Enhanced optimization failed")


[INFO] Initializing Enhanced VRP Optimizer with distance minimization...
[INFO] Enhanced input data validation completed successfully
[INFO] Using specified depot location: 08020
[INFO] Depot return enabled: False
[INFO] Maximum orders per truck: 3
[INFO] Enhanced optimizer initialized:
[INFO]   Orders: 5, Trucks: 5, Locations: 6
[INFO]   Depot location: 08020
[INFO]   Total order volume: 150.0 m³
[INFO]   Total truck capacity: 225.0 m³


🚀 Running Enhanced MILP optimization...


AttributeError: 'EnhancedVrpOptimizer' object has no attribute 'set_weights'

## Genetic Algorithm

The Genetic Algorithm optimizer uses evolutionary metaheuristics for large-scale problems.


In [ ]:
print("🧬 Running Genetic Algorithm optimization...")

# Create genetic optimizer
genetic_optimizer = GeneticVrpOptimizer(
    orders_df=orders_df,
    trucks_df=trucks_df,
    distance_matrix=distance_matrix,
    depot_location='08020',
    depot_return=False,
    max_orders_per_truck=3
)

# Set genetic algorithm parameters
genetic_optimizer.set_parameters(
    population_size=50,
    max_generations=30,
    mutation_rate=0.1
)

# Run optimization
start_time = time.time()
genetic_solution = genetic_optimizer.optimize()
solve_time = time.time() - start_time

if genetic_solution:
    print(f"✅ Genetic optimization successful! ({solve_time:.2f}s)")
    print(f"💰 Total cost: €{genetic_solution['costs']['total_cost']:.0f}")
    print(f"🚛 Trucks used: {len(genetic_solution['selected_trucks'])}")
    print(f"📦 Orders delivered: {len(genetic_solution['assignments_df'])}")
    print(f"🔄 Generations run: {genetic_optimizer.generation}")
    
    # Display solution summary
    print("\n📊 Genetic Solution Summary:")
    print(genetic_optimizer.get_solution_summary_text())
else:
    print("❌ Genetic optimization failed")


## Custom Data Example

Let's create a custom problem with our own data.


In [ ]:
print("📝 Creating custom data example...")

# Create custom orders
custom_orders = {
    'order_id': ['O1', 'O2', 'O3', 'O4', 'O5'],
    'volume': [2.5, 1.8, 3.2, 2.1, 1.5],
    'postal_code': ['08020', '08027', '08030', '08035', '08040']
}
custom_orders_df = pd.DataFrame(custom_orders)

# Create custom trucks
custom_trucks = {
    'truck_id': ['T1', 'T2', 'T3'],
    'capacity': [8.0, 10.0, 12.0],
    'cost': [400, 500, 600]
}
custom_trucks_df = pd.DataFrame(custom_trucks)

# Calculate distance matrix
custom_postal_codes = custom_orders_df['postal_code'].unique().tolist()
custom_distance_matrix = calculate_distance_matrix(custom_postal_codes)

print("📦 Custom Orders:")
display(custom_orders_df)

print("\n🚛 Custom Trucks:")
display(custom_trucks_df)

print("\n🗺️ Custom Distance Matrix:")
display(custom_distance_matrix)


In [ ]:
# Optimize custom data
print("🚀 Optimizing custom data...")

custom_optimizer = VrpOptimizer(
    orders_df=custom_orders_df,
    trucks_df=custom_trucks_df,
    distance_matrix=custom_distance_matrix,
    depot_location='08020',
    depot_return=True,  # Trucks must return to depot
    max_orders_per_truck=2,
    enable_greedy_routes=True
)

custom_optimizer.build_model()
success = custom_optimizer.solve(timeout=300)

if success:
    custom_solution = custom_optimizer.get_solution()
    print(f"✅ Custom optimization successful!")
    print(f"💰 Total cost: €{custom_solution['costs']['total_cost']:.0f}")
    print(f"🚛 Trucks used: {custom_solution['selected_trucks']}")
    
    # Show assignments
    print("\n📋 Order Assignments:")
    for assignment in custom_solution['assignments']:
        print(f"  {assignment['order_id']} → {assignment['truck_id']}")
else:
    print("❌ Custom optimization failed")


## Performance Comparison

Let's compare the performance of all three methods.


In [ ]:
print("📊 Performance Comparison")
print("=" * 50)

results = []

# Standard MILP + Greedy
print("\n🚀 Standard MILP + Greedy...")
start_time = time.time()
std_optimizer = VrpOptimizer(orders_df, trucks_df, distance_matrix)
std_optimizer.build_model()
std_success = std_optimizer.solve(timeout=300)
std_time = time.time() - start_time

if std_success:
    std_solution = std_optimizer.get_solution()
    results.append({
        'Method': 'Standard MILP + Greedy',
        'Cost (€)': std_solution['costs']['total_cost'],
        'Trucks': len(std_solution['selected_trucks']),
        'Time (s)': std_time,
        'Success': '✅'
    })

# Enhanced MILP
print("\n🚀 Enhanced MILP...")
start_time = time.time()
enh_optimizer = EnhancedVrpOptimizer(orders_df, trucks_df, distance_matrix)
enh_optimizer.set_weights(cost_weight=0.7, distance_weight=0.3)
enh_optimizer.build_model()
enh_success = enh_optimizer.solve(timeout=300)
enh_time = time.time() - start_time

if enh_success:
    enh_solution = enh_optimizer.get_solution()
    results.append({
        'Method': 'Enhanced MILP',
        'Cost (€)': enh_solution['costs']['total_cost'],
        'Trucks': len(enh_solution['selected_trucks']),
        'Time (s)': enh_time,
        'Success': '✅'
    })

# Genetic Algorithm
print("\n🧬 Genetic Algorithm...")
start_time = time.time()
gen_optimizer = GeneticVrpOptimizer(orders_df, trucks_df, distance_matrix)
gen_optimizer.set_parameters(population_size=30, max_generations=20, mutation_rate=0.1)
gen_solution = gen_optimizer.optimize()
gen_time = time.time() - start_time

if gen_solution:
    results.append({
        'Method': 'Genetic Algorithm',
        'Cost (€)': gen_solution['costs']['total_cost'],
        'Trucks': len(gen_solution['selected_trucks']),
        'Time (s)': gen_time,
        'Success': '✅'
    })

# Display results
results_df = pd.DataFrame(results)
print("\n📊 Results Summary:")
display(results_df)

# Find best method
best_cost = results_df['Cost (€)'].min()
best_method = results_df[results_df['Cost (€)'] == best_cost]['Method'].iloc[0]
print(f"\n🏆 Best cost: €{best_cost:.0f} ({best_method})")


## Visualization

Let's create some visualizations of our results.


In [ ]:
# Plot routes for the best solution
if std_success:
    print("🗺️ Creating route visualization...")
    
    fig = plot_routes(
        routes_df=std_solution['routes_df'],
        orders_df=orders_df,
        trucks_df=trucks_df,
        distance_matrix=distance_matrix,
        figsize=(12, 8)
    )
    
    plt.title("Standard MILP + Greedy - Route Visualization")
    plt.show()
    
    # Plot costs
    print("💰 Creating cost visualization...")
    
    fig = plot_costs(
        cost_summary=std_solution['costs'],
        trucks_df=trucks_df,
        figsize=(10, 6)
    )
    
    plt.title("Cost Breakdown")
    plt.show()
    
    # Plot utilization
    print("📊 Creating utilization visualization...")
    
    fig = plot_utilization(
        utilization_data=std_solution['utilization'],
        trucks_df=trucks_df,
        figsize=(12, 8)
    )
    
    plt.title("Truck Utilization")
    plt.show()
else:
    print("❌ No solution available for visualization")


## Summary

This notebook demonstrated:

1. **Basic Usage**: How to generate data and run optimization
2. **Standard MILP + Greedy**: Two-phase hybrid optimization
3. **Enhanced MILP**: Multi-objective optimization with weighting
4. **Genetic Algorithm**: Evolutionary metaheuristic approach
5. **Custom Data**: Using your own orders and trucks
6. **Performance Comparison**: Comparing all three methods
7. **Visualization**: Creating plots of routes, costs, and utilization

### Key Takeaways:

- **Standard MILP + Greedy**: Fast and reliable for most problems
- **Enhanced MILP**: Best for optimal solutions with cost-distance trade-offs
- **Genetic Algorithm**: Good for large-scale problems and exploration
- **Custom Data**: Easy to use your own orders and trucks
- **Visualization**: Built-in plotting functions for analysis

### Next Steps:

- Try different parameter settings
- Experiment with larger datasets
- Use real-world distances for geographic accuracy
- Integrate with your own data sources
